载入特征文件（）并建立网络，分类，进行训练，

In [1]:
import h5py
import numpy as np
from sklearn.utils import shuffle
np.random.seed(0)

In [2]:
train_features = []
test_features = []
train_labels = []

with h5py.File("xception_bottleneck.h5", 'r') as h:
    train_features.append(np.array(h['train_features']))
    test_features.append(np.array(h['test_features']))
    train_labels = np.array(h['train_label'])

In [3]:
train_features = train_features[0]
test_features = test_features[0]

In [4]:
#验证数据集的形状是否正确
print("train_features shape:",np.shape(train_features))
print("test_features shape:",np.shape(test_features))
print("train_labels shape:",np.shape(train_labels))

train_features shape: (2002, 2048)
test_features shape: (1000, 2048)
train_labels shape: (2002,)


In [5]:
#是否打乱
train_features, train_labels = shuffle(train_features, train_labels)

构建网络

In [6]:
from keras.layers import Input, Dropout, Dense
from keras.models import Model
from keras import optimizers
from keras.callbacks import TensorBoard

Using TensorFlow backend.


In [7]:
inputlayer = Input(shape=(train_features.shape[1:]))

In [8]:
x = inputlayer

x = Dropout(0.5)(x)

#x = Dense(512, activation='relu')(x)
#x = Dropout(0.85)(x)
outlayer = Dense(1, activation='sigmoid')(x)

model = Model(input=inputlayer, output=outlayer)

/Users/liujiaye/anaconda3/envs/tensorflow2/lib/python3.6/site-packages/ipykernel_launcher.py:9: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`
  if __name__ == '__main__':


In [9]:
#优化器参考：https://keras.io/optimizers/
opt = optimizers.SGD(lr=0.01, momentum=0.9)
model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
model.fit(train_features,train_labels,nb_epoch=200,verbose=1,validation_split=0.2, batch_size=128, callbacks=[TensorBoard(log_dir='./log')])
model.save('model.h5')

/Users/liujiaye/anaconda3/envs/tensorflow2/lib/python3.6/site-packages/ipykernel_launcher.py:4: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  after removing the cwd from sys.path.


Train on 1601 samples, validate on 401 samples
Epoch 1/200
1601/1601 [==============================] - 0s 72us/step - loss: 0.6956 - acc: 0.5365 - val_loss: 0.6782 - val_acc: 0.6085
Epoch 2/200
1601/1601 [==============================] - 0s 60us/step - loss: 0.6779 - acc: 0.5628 - val_loss: 0.6658 - val_acc: 0.6334
Epoch 3/200
1601/1601 [==============================] - 0s 57us/step - loss: 0.6722 - acc: 0.5728 - val_loss: 0.6556 - val_acc: 0.6359
Epoch 4/200
1601/1601 [==============================] - 0s 58us/step - loss: 0.6699 - acc: 0.5946 - val_loss: 0.6514 - val_acc: 0.6334
Epoch 5/200
1601/1601 [==============================] - 0s 49us/step - loss: 0.6586 - acc: 0.6165 - val_loss: 0.6569 - val_acc: 0.6110
Epoch 6/200
1601/1601 [==============================] - 0s 53us/step - loss: 0.6607 - acc: 0.6040 - val_loss: 0.6474 - val_acc: 0.6359
Epoch 7/200
1601/1601 [==============================] - 0s 47us/step - loss: 0.6601 - acc: 0.6052 - val_loss: 0.6535 - val_acc: 0.6259
E

1601/1601 [==============================] - 0s 75us/step - loss: 0.6582 - acc: 0.6065 - val_loss: 0.6697 - val_acc: 0.5761
Epoch 61/200
1601/1601 [==============================] - 0s 73us/step - loss: 0.6585 - acc: 0.6121 - val_loss: 0.6517 - val_acc: 0.6135
Epoch 62/200
1601/1601 [==============================] - 0s 72us/step - loss: 0.6493 - acc: 0.6240 - val_loss: 0.6586 - val_acc: 0.5960
Epoch 63/200
1601/1601 [==============================] - 0s 67us/step - loss: 0.6495 - acc: 0.6271 - val_loss: 0.6500 - val_acc: 0.6185
Epoch 64/200
1601/1601 [==============================] - 0s 74us/step - loss: 0.6597 - acc: 0.6171 - val_loss: 0.6441 - val_acc: 0.6209
Epoch 65/200
1601/1601 [==============================] - 0s 68us/step - loss: 0.6568 - acc: 0.6152 - val_loss: 0.6435 - val_acc: 0.6334
Epoch 66/200
1601/1601 [==============================] - 0s 78us/step - loss: 0.6479 - acc: 0.6384 - val_loss: 0.6515 - val_acc: 0.6110
Epoch 67/200
1601/1601 [==============================

Epoch 120/200
1601/1601 [==============================] - 0s 70us/step - loss: 0.6494 - acc: 0.6159 - val_loss: 0.6457 - val_acc: 0.6309
Epoch 121/200
1601/1601 [==============================] - 0s 62us/step - loss: 0.6484 - acc: 0.6196 - val_loss: 0.6434 - val_acc: 0.6359
Epoch 122/200
1601/1601 [==============================] - 0s 62us/step - loss: 0.6514 - acc: 0.6196 - val_loss: 0.6418 - val_acc: 0.6384
Epoch 123/200
1601/1601 [==============================] - 0s 64us/step - loss: 0.6488 - acc: 0.6296 - val_loss: 0.6417 - val_acc: 0.6259
Epoch 124/200
1601/1601 [==============================] - 0s 58us/step - loss: 0.6583 - acc: 0.6140 - val_loss: 0.6500 - val_acc: 0.6110
Epoch 125/200
1601/1601 [==============================] - 0s 71us/step - loss: 0.6501 - acc: 0.6190 - val_loss: 0.6534 - val_acc: 0.6135
Epoch 126/200
1601/1601 [==============================] - 0s 67us/step - loss: 0.6508 - acc: 0.6371 - val_loss: 0.6446 - val_acc: 0.6409
Epoch 127/200
1601/1601 [=========

1601/1601 [==============================] - 0s 61us/step - loss: 0.6537 - acc: 0.6246 - val_loss: 0.6414 - val_acc: 0.6259
Epoch 180/200
1601/1601 [==============================] - 0s 76us/step - loss: 0.6546 - acc: 0.6240 - val_loss: 0.6420 - val_acc: 0.6434
Epoch 181/200
1601/1601 [==============================] - 0s 73us/step - loss: 0.6524 - acc: 0.6302 - val_loss: 0.6422 - val_acc: 0.6434
Epoch 182/200
1601/1601 [==============================] - 0s 74us/step - loss: 0.6454 - acc: 0.6171 - val_loss: 0.6441 - val_acc: 0.6409
Epoch 183/200
1601/1601 [==============================] - 0s 68us/step - loss: 0.6487 - acc: 0.6252 - val_loss: 0.6560 - val_acc: 0.6060
Epoch 184/200
1601/1601 [==============================] - 0s 75us/step - loss: 0.6488 - acc: 0.6296 - val_loss: 0.6518 - val_acc: 0.6085
Epoch 185/200
1601/1601 [==============================] - 0s 81us/step - loss: 0.6520 - acc: 0.6346 - val_loss: 0.6546 - val_acc: 0.6035
Epoch 186/200
1601/1601 [=======================

In [10]:
pred = model.predict(test_features, batch_size=None, verbose=0, steps=None)
pred = pred.clip(min=0.005, max=0.995)

In [11]:
print(pred[:20])

[[ 0.60811883]
 [ 0.35920987]
 [ 0.41625449]
 [ 0.65608013]
 [ 0.35209915]
 [ 0.55895132]
 [ 0.62674665]
 [ 0.41977066]
 [ 0.70954937]
 [ 0.6066907 ]
 [ 0.361211  ]
 [ 0.47873423]
 [ 0.72412759]
 [ 0.37034383]
 [ 0.78247076]
 [ 0.50927764]
 [ 0.53015971]
 [ 0.81270283]
 [ 0.48005006]
 [ 0.5317378 ]]
